## Segment Anything Model (SAM) - EELS: live workflow
- Acquire an image using HAADF detector
- Choose a point 
- Acquire EELS detector signal
#### Contributor(s): Utkarsh Pratiush <utkarshp1161@gmail.com> - 2nd May 2025
#### edited - 
   

In [ ]:
import os
import json
import pickle
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from typing import Dict, List
from autoscript_tem_microscope_client import TemMicroscopeClient

from stemOrchestrator.logging_config import setup_logging
from stemOrchestrator.acquisition import TFacquisition
from stemOrchestrator.process import HAADF_tiff_to_png
from stemOrchestrator.simulation import DMtwin

In [ ]:
def acquire_and_plot_eels(
    image_data: np.ndarray,
    point_idx: int,
    x: float,
    y: float,
    eels_exposure: float,
    eels_offset: float,
    out_path: str,
    tf_acquisition,
    dm_acquisition,
):
    print(
        f"\n>> Acquiring EELS at normalized position: ({x}, {y}) for point {point_idx}"
    )

    directory = os.path.join(out_path, f"manual_point_{point_idx}")
    os.makedirs(directory, exist_ok=True)

    # Move and acquire at centroid
    tf_acquisition.move_paused_beam(x, y)
    tf_acquisition.unblank_beam()
    dm_acquisition.acquire_camera(exposure=eels_exposure)
    tf_acquisition.blank_beam()

    array_list, shape, dtype = dm_acquisition.get_eels()
    spec_eels = np.array(array_list, dtype=dtype).reshape(shape)

    # Query beam position
    position = tf_acquisition.query_paused_beam_positon()
    x_actual = position.x
    y_actual = position.y
    formatted_position = f"({x_actual:.2g}, {y_actual:.2g})"

    # Plot image + spectrum
    fig, axs = plt.subplots(1, 2, figsize=(18, 6))
    axs[0].imshow(image_data, cmap="gray")
    axs[0].set_title("HAADF with EELS Position")
    axs[0].scatter(
        x * image_data.shape[1],
        y * image_data.shape[0],
        c="r",
        s=100,
        marker="x",
        label=f"Position: {formatted_position}",
    )
    axs[0].axis("off")

    axs[1].plot(np.arange(len(spec_eels)) - eels_offset, spec_eels)
    axs[1].set_title("EELS Spectrum")
    axs[1].set_xlabel("Energy loss (eV)")
    axs[1].set_ylabel("Counts")

    plt.tight_layout()
    fig.savefig(os.path.join(directory, f"haadf_eels_point_{point_idx}.png"), dpi=300)
    plt.close()

    np.save(os.path.join(directory, f"eels_spectrum_point_{point_idx}.npy"), spec_eels)

    # Save metadata
    metadata = {
        "normalized_position": [x, y],
        "beam_position": [x_actual, y_actual],
        "exposure_time": eels_exposure,
    }
    with open(os.path.join(directory, "metadata.json"), "w") as f:
        json.dump(metadata, f, indent=2)

    print(f"Saved EELS spectrum and image overlay for point {point_idx}")

In [ ]:
def main(config: Dict):
    ip = config["ip"]
    port = config["port"]
    haadf_exposure = config["haadf_exposure"]
    haadf_resolution = config["haadf_resolution"]
    out_path = config["out_path"]
    eels_exposure = config["eels_exposure"]
    eels_offset = config["eels_offset"]

    setup_logging(out_path)

    microscope = TemMicroscopeClient()
    microscope.connect(ip, port=port)
    tf_acquisition = TFacquisition(microscope=microscope, offline=True)
    dm_acquisition = DMtwin()

    # Acquire HAADF
    haadf_array, haadf_tiff_name = tf_acquisition.acquire_haadf(
        exposure=haadf_exposure, resolution=haadf_resolution
    )
    HAADF_tiff_to_png(haadf_tiff_name)
    print("HAADF acquisition complete.")

    # Save exposure settings
    spec_metadata_dict = {
        "eels_exposure_sec": eels_exposure,
        "eels_offset": eels_offset,
    }
    with open(os.path.join(out_path, "spectrometer_metadata_dict.pkl"), "wb") as file:
        pickle.dump(spec_metadata_dict, file)

    # User-defined positions
    print("\n--- Enter normalized (x, y) positions for EELS ---")
    print("Format: 0.5,0.6   (press Enter to stop)\n")

    point_idx = 0
    eels_positions = []

    while True:
        inp = input(f"Enter position {point_idx + 1}: ").strip()
        if inp == "":
            break
        try:
            x, y = map(float, inp.split(","))
            if not (0 <= x <= 1 and 0 <= y <= 1):
                print("Coordinates must be between 0 and 1.")
                continue
        except ValueError:
            print("Invalid format. Use: 0.5,0.6")
            continue

        acquire_and_plot_eels(
            haadf_array,
            point_idx,
            x,
            y,
            eels_exposure,
            eels_offset,
            out_path,
            tf_acquisition,
            dm_acquisition,
        )
        eels_positions.append((point_idx, x, y))
        point_idx += 1

    # Final summary HAADF plot
    if len(eels_positions) > 0:
        fig, ax = plt.subplots(figsize=(8, 8))
        ax.imshow(haadf_array, cmap="gray")
        ax.set_title("HAADF with All EELS Points")
        ax.axis("off")

        for idx, x, y in eels_positions:
            ax.scatter(
                x * haadf_array.shape[1],
                y * haadf_array.shape[0],
                c="r",
                s=100,
                marker="x",
                label=f"Pt {idx}",
            )

        plt.tight_layout()
        plt.savefig(os.path.join(out_path, "HAADF_all_EELS_positions.png"), dpi=300)
        plt.close()
        print("✔ Saved summary plot of all EELS points.")

    print("\n EELS acquisition complete.")

In [ ]:
ip = os.getenv("MICROSCOPE_IP")
port = os.getenv("MICROSCOPE_PORT")

if not ip or not port:
    secret_path = Path("../../config_secret.json")
    if secret_path.exists():
        with open(secret_path, "r") as f:
            secret = json.load(f)
            ip = ip or secret.get("ip_TF_sim")
            port = port or secret.get("port_TF_sim")

if not ip:
    ip = input("Enter microscope IP: ")
if not port:
    port = input("Enter microscope Port: ")
port = int(port)

config = {
    "ip": ip,
    "port": port,
    "haadf_exposure": 40e-8,
    "haadf_resolution": 512,
    "eels_exposure": 0.5,  # in seconds
    "eels_offset": 0,  # energy offset for plotting
    "out_path": ".",
}

main(config=config)